### CNN Feature Based Image Copy Detection With Contextual Hash Embeddings
paper [here](https://www.mdpi.com/2227-7390/8/7/1172/pdf)

In [1]:
from google.colab import drive
drive.mount("/content/gdrive")

Mounted at /content/gdrive


In [2]:
def hashing(x):
  ls1=[]
  for i in range(0,len(x)):
    for j in range(0,len(x)):
      string=""
      try:
        a=x[i,:][j]
        if a >  x[i-1,:][j-1] and i-1 != -1 and j-1 != -1: # upper left
          string=string+str(1)
        else:
          string=string+str(0)
      except:
        string=string+str(0)
        pass
      try:
        a=x[i,:][j]
        if a > x[i-1,:][j] and i-1 != -1: # upper 
          string=string+str(1)
        else:
          string=string+str(0)
      except:
        string=string+str(0)
        pass
      try:
        a=x[i,:][j]
        if a > x[i-1,:][j+1]  and i-1 != -1:# upper right
          string=string+str(1)
        else:
          string=string+str(0)
      except:
        string=string+str(0)
        pass
      try:
        a=x[i,:][j]
        if a > x[i,:][j+1]: # right
          string=string+str(1)
        else:
          string=string+str(0)
      except:
        string=string+str(0)
        pass
      try:
        a=x[i,:][j]
        if a > x[i+1,:][j+1]: #bottom right
          string=string+str(1)
        else:
          string=string+str(0)
      except:
        string=string+str(0)
        pass
      try:
        a=x[i,:][j]
        if a > x[i+1,:][j]: # bottom
          string=string+str(1)
        else:
          string=string+str(0)
      except:
        string=string+str(0)
        pass
      try:
        a=x[i,:][j]
        if a > x[i+1,:][j-1] and j-1 != -1: #bottom left
          string=string+str(1)
        else:
          string=string+str(0)
      except:
        string=string+str(0)
        pass
      try:
        a=x[i,:][j]
        if a > x[i,:][j-1] and j-1 != -1:  # left
          string=string+str(1)
        else:
          string=string+str(0)
      except:
        string=string+str(0)
        pass
      ls1.append(string)
  return ls1


In [2]:
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms

import os
import cv2
import numpy as np
import sklearn
from sklearn import preprocessing

In [3]:
#using Pretrained model
import torchvision
from torchvision import models
model = models.alexnet(pretrained = True)

Downloading: "https://download.pytorch.org/models/alexnet-owt-4df8aa71.pth" to /root/.cache/torch/hub/checkpoints/alexnet-owt-4df8aa71.pth


In [4]:
for child in model.children():
    print(child)
    break

Sequential(
  (0): Conv2d(3, 64, kernel_size=(11, 11), stride=(4, 4), padding=(2, 2))
  (1): ReLU(inplace=True)
  (2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  (3): Conv2d(64, 192, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
  (4): ReLU(inplace=True)
  (5): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  (6): Conv2d(192, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (7): ReLU(inplace=True)
  (8): Conv2d(384, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (9): ReLU(inplace=True)
  (10): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (11): ReLU(inplace=True)
  (12): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
)


In [5]:
#taking only Conv LAyers
model1 = child[:12]
model1 = model1.eval()

In [6]:
model1 #model Architecture # taking 5th layer

Sequential(
  (0): Conv2d(3, 64, kernel_size=(11, 11), stride=(4, 4), padding=(2, 2))
  (1): ReLU(inplace=True)
  (2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  (3): Conv2d(64, 192, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
  (4): ReLU(inplace=True)
  (5): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  (6): Conv2d(192, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (7): ReLU(inplace=True)
  (8): Conv2d(384, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (9): ReLU(inplace=True)
  (10): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (11): ReLU(inplace=True)
)

In [7]:
def get_hash_embedding(x):
    binary_coded = []
    adj = 8
    for row in range(0,x.shape[0]):
        for col in range(0,x.shape[1]):
            x[row][col]
            if(row == 0 and col == 0):
                encode_num = []
                if(x[row][col] < x[row+1][col]):
                    encode_num.append(str(1))
                else:
                    encode_num.append(str(0))
                if(x[row][col] < x[row+1][col+1]):
                    encode_num.append(str(1))
                else:
                    encode_num.append(str(0))
                if(x[row][col] < x[row][col+1]):
                    encode_num.append(str(1))
                else:
                    encode_num.append(str(0))
                difference=adj-len(encode_num)
                for num in range(0,difference):
                    encode_num.append(str(0))
                binary_coded.append(''.join(encode_num))
            elif(row == 0 and col == x.shape[1]-1):
                encode_num = []
                if(x[row][col] < x[row][col-1]):
                    encode_num.append("1")
                else:
                    encode_num.append("0")
                if(x[row][col] < x[row+1][col-1]):
                    encode_num.append("1")
                else:
                    encode_num.append("0")
                if(x[row][col] < x[row+1][col]):
                    encode_num.append("1")
                else:
                    encode_num.append("0")
                difference=adj-len(encode_num)
                for num in range(0,difference):
                    encode_num.append(str(0))
                binary_coded.append(''.join(encode_num))
            elif(row == x.shape[0]-1 and col == 0):
                encode_num = []
                if(x[row][col] < x[row-1][col]):
                    encode_num.append("1")
                else:
                    encode_num.append("0")
                encode_num.append("0")
                encode_num.append("0")
                encode_num.append("0")
                encode_num.append("0")
                encode_num.append("0")
                if(x[row][col] < x[row][col+1]):
                    encode_num.append("1")
                else:
                    encode_num.append("0")
                if(x[row][col] < x[row-1][col+1]):
                    encode_num.append("1")
                else:
                    encode_num.append("0")
                binary_coded.append(''.join(encode_num))
            elif(row == x.shape[0]-1 and col == x.shape[1]-1):
                encode_num = []
                if(x[row][col] < x[row-1][col]):
                    encode_num.append("1")
                else:
                    encode_num.append("0")
                if(x[row][col] < x[row-1][col-1]):
                    encode_num.append("1")
                else:
                    encode_num.append("0")
                if(x[row][col] < x[row][col-1]):
                    encode_num.append("1")
                else:
                    encode_num.append("0")
                difference=adj-len(encode_num)
                for num in range(0,difference):
                    encode_num.append(str(0))
                binary_coded.append(''.join(encode_num))
            elif(col == 0 and row !=0):
                encode_num = []
                if(x[row][col] < x[row-1][col]):
                    encode_num.append("1")
                else:
                    encode_num.append("0")
                encode_num.append("0")
                encode_num.append("0")
                encode_num.append("0")
                if(x[row][col] < x[row+1][col]):
                    encode_num.append("1")
                else:
                    encode_num.append("0")
                if(x[row][col] < x[row+1][col+1]):
                    encode_num.append("1")
                else:
                    encode_num.append("0")
                if(x[row][col] < x[row][col+1]):
                    encode_num.append("1")
                else:
                    encode_num.append("0")
                if(x[row][col] < x[row-1][col+1]):
                    encode_num.append("1")
                else:
                    encode_num.append("0")
                binary_coded.append(''.join(encode_num))
            #condition 2
            elif(row == 0 and col !=0):
                encode_num = []
                encode_num.append("0")
                encode_num.append("0")
                if(x[row][col] < x[row][col-1]):
                    encode_num.append("1")
                else:
                    encode_num.append("0")
                if(x[row][col] < x[row+1][col-1]):
                    encode_num.append("1")
                else:
                    encode_num.append("0")
                if(x[row][col] < x[row+1][col]):
                    encode_num.append("1")
                else:
                    encode_num.append("0")
                if(x[row][col] < x[row+1][col+1]):
                    encode_num.append("1")
                else:
                    encode_num.append("0")
                if(x[row][col] < x[row][col+1]):
                    encode_num.append("1")
                else:
                    encode_num.append("0")
                encode_num.append("0")
                binary_coded.append(''.join(encode_num))
            #condition 3
            elif(row == x.shape[0]-1  and col != x.shape[1]-1):
                encode_num = []
                if(x[row][col] < x[row-1][col]):
                    encode_num.append("1")
                else:
                    encode_num.append("0")
                if(x[row][col] < x[row-1][col-1]):
                    encode_num.append("1")
                else:
                    encode_num.append("0")
                if(x[row][col] < x[row][col-1]):
                    encode_num.append("1")
                else:
                    encode_num.append("0")
                encode_num.append("0")
                encode_num.append("0")
                encode_num.append("0")
                if(x[row][col] < x[row][col+1]):
                    encode_num.append("1")
                else:
                    encode_num.append("0")
                if(x[row][col] < x[row-1][col+1]):
                    encode_num.append("1")
                else:
                    encode_num.append("0")
                binary_coded.append(''.join(encode_num))
            #condition 4
            elif(row != x.shape[0]-1  and col == x.shape[1]-1):
                encode_num = []
                if(x[row][col] < x[row-1][col]):
                    encode_num.append("1")
                else:
                    encode_num.append("0")
                if(x[row][col] < x[row-1][col-1]):
                    encode_num.append("1")
                else:
                    encode_num.append("0")
                if(x[row][col] < x[row][col-1]):
                    encode_num.append("1")
                else:
                    encode_num.append("0")
                if(x[row][col] < x[row+1][col-1]):
                    encode_num.append("1")
                else:
                    encode_num.append("0")
                if(x[row][col] < x[row+1][col]):
                    encode_num.append("1")
                else:
                    encode_num.append("0")
                encode_num.append("0")
                encode_num.append("0")
                encode_num.append("0")
                binary_coded.append(''.join(encode_num))
            #condition 5
            else:
                encode_num = []
                if(x[row][col] < x[row-1][col]):
                    encode_num.append("1")
                else:
                    encode_num.append("0")
                if(x[row][col] < x[row-1][col-1]):
                    encode_num.append("1")
                else:
                    encode_num.append("0")
                if(x[row][col] < x[row][col-1]):
                    encode_num.append("1")
                else:
                    encode_num.append("0")
                if(x[row][col] < x[row+1][col-1]):
                    encode_num.append("1")
                else:
                    encode_num.append("0")
                if(x[row][col] < x[row+1][col]):
                    encode_num.append("1")
                else:
                    encode_num.append("0")
                if(x[row][col] < x[row+1][col+1]):
                    encode_num.append("1")
                else:
                    encode_num.append("0")
                if(x[row][col] < x[row][col+1]):
                    encode_num.append("1")
                else:
                    encode_num.append("0")
                if(x[row][col] < x[row-1][col+1]):
                    encode_num.append("1")
                else:
                    encode_num.append("0")
                binary_coded.append(''.join(encode_num))
    return binary_coded

In [8]:
#Dividing Image into Slices
def get_blocks(cord,radius,path):
    max_loc = cord
    circle_radius = radius

    img = cv2.imread(path)
    
    image_array = img.copy()
    mask = np.zeros(image_array.shape, dtype=np.uint8)
    cv2.circle(mask, max_loc, circle_radius, (255, 255, 255), -1, 8, 0)

    result_array = image_array & mask

    # Crop/center result (assuming max_loc is of the form (x, y))
    result_array1 = result_array[max(0,max_loc[1] - circle_radius):max(0,max_loc[1] + circle_radius),
                             max(0,max_loc[0] - circle_radius):max(0,max_loc[0] + circle_radius)]

    #convert to log polar space
    log_polar = cv2.logPolar(result_array1,(radius,radius),circle_radius,1)

    
    #divide into bllocks
    log_polar_gray  = log_polar
    M = int(log_polar_gray.shape[0]/4)
    N = int(log_polar_gray.shape[1]/4)

    slices = []
    for j in range(4):
        for i in range(4):
            if j!=3 and i!=3: #center
                slices.append(img[j*N:j*N+N,i*M:i*M+M])
            if j!=3 and i == 3: #top right corner
                slices.append(img[j*N:j*N+N,i*M:])
            if j == 3 and i!=3: #bottom down
                slices.append(img[j*N:,i*M:i*M+M])
            if j==3 and i ==3: #bottom corner
                slices.append(img[j*N:,i*M:])
    
    return slices
    

In [9]:
#for getting keypoints and radius
def get_coordinates_and_radius(keypoints_orb):
    keys = []
    for i in keypoints_orb:
        cord = [int(i) for i in i.pt]
        rad = int(i.size/2)
        keys.append([tuple(cord),rad])
    return keys

### CFM_LocalCNNFeatures_Hashembedding

In [13]:
files_path="gdrive/My Drive/copy_detection/few_images/"

val =sorted([int(i.split(".")[0]) for i in os.listdir(files_path)]) #os.listdir wont give you file order
images = [str(i)+".jpg" for i in val] #getting files in order

total_images_features = []
total_images_hash_values = []

for i in images:
    #reading image
    img = cv2.imread(files_path+i)
    img = cv2.cvtColor(img,cv2.COLOR_BGR2RGB)
    
    #image to alexnet to get CFMs
    data_transform = transforms.ToTensor()
    img1 = data_transform(img)
    img2 = img1.unsqueeze(0)
    op = model1(img2)
    output = np.array(op.detach().numpy()[0]) #1,256,height,width
    
    #calculating image and cfm areas for ratio
    img_area = img.shape[0]*img.shape[1]
    cfm_area = output.shape[1]*output.shape[2] 
    area_ratio = cfm_area/img_area
    
    #passing image to ORB
    orb = cv2.ORB_create(nfeatures=1500)
    keypoints_orb, descriptors = orb.detectAndCompute(img, None)
    keypoints = get_coordinates_and_radius(keypoints_orb)
    
    
    #Scaling CFM radius and coordinates
    cfm_radius = []
    cfm_coo = []
    for key in keypoints:
        cfm_radius.append(np.float32(area_ratio*key[1]))
        
        cfm_x = (output.shape[1]*key[0][0])/img.shape[0]
        cfm_y = (output.shape[2]*key[0][1])/img.shape[1]
        cfm_coo.append((np.float32(cfm_x),np.float32(cfm_y)))

    #Finding normalized radius from Minimum of 1 to 8 value
    val = [key[1] for key in keypoints]    
    x = np.array(cfm_radius)
    b = (max(val)-min(val))/(min(val))
    a = 1
    normalized_radius=(b-a)*((x-x.min())/(x.max()-x.min()))+a
    
    #For finding local CNN features(mapping keypoints to CFM)
    all_keypoint_features = []
    output = np.array(op.detach().numpy()[0])
    for cord,rad in zip(cfm_coo,normalized_radius):
        feature_map = []
        for fmap in range(len(output)):
            op = np.sum(output[fmap][int(cord[0]):int(cord[0])+int(rad),int(cord[1]):int(cord[1])+int(rad)])
            feature_map.append(op)
        op1 = np.array(feature_map).reshape(1,256)
        norm_final = sklearn.preprocessing.normalize(op1, norm='l2', axis=1, copy=True, return_norm=False)
        all_keypoint_features.append(norm_final)    
    
    #stacking all keypoints(ex:400) of a single image
    keypoint_feat = all_keypoint_features[0]
    for keypoint in all_keypoint_features[1:]:
        keypoint_feat = np.vstack((keypoint_feat,keypoint))
    
    print(keypoint_feat.shape)#(483,256)(500,256) 256 columns fixed
    total_images_features.append(keypoint_feat)
    
    #multiplying radius with Alpha 3.2
    expanded_radius = []
    for key,radius in keypoints:
        expand_r = int(int(radius/2)*3.2)
        expanded_radius.append(expand_r)
    
    cords = []
    for cord,radius in keypoints:
        cords.append(cord)
    
    #get Hash Values
    all_hash_values = []
    for cord,ex_radius in zip(cords,expanded_radius):
        slices = get_blocks(cord,radius,files_path+i)
        grey_intensities = np.array([np.mean(i) for i in slices]).reshape(4,4)
        hash_val = get_hash_embedding(grey_intensities)
        #hash_val = hashing(grey_intensities)        
        all_hash_values.append(hash_val)
    
    total_images_hash_values.append(all_hash_values)

(1500, 256)
(1500, 256)
(1500, 256)
(307, 256)
(1500, 256)


In [14]:
#stacking all images all keypoints in to a single vector
img_feat = total_images_features[0]
for i in total_images_features[1:]:
    img_feat = np.vstack((img_feat,i))
    
print(img_feat.shape)

(6307, 256)


In [15]:
np.sqrt(256*27402) #as per literature reading

2648.567914930633

In [16]:
  total_images_features[0].shape #no of keypoints for first image, Keypoints may vary for image to image

(1500, 256)

In [17]:
len(total_images_hash_values[1]) #same no of hash values as keypoints for all images

1500

In [18]:
len(total_images_hash_values) # hash values of 38 images

5

### 3. Local_CNN_VBOW

In [19]:
import scipy
import random

random.seed(1234)

from scipy.cluster.vq import kmeans,vq #vq stands for vector quantization
k= 100 #maximum clusters can be 50
                                           

voc1,variance=kmeans(img_feat,k,100)        #voc decides no of clusters to be formed, based on distace
                                           # 50 is no of iterations

#clustering all the keypoints of all images
database_words = []
im_features=np.zeros((len(os.listdir(files_path)),k),"float32")
for i in range(len(os.listdir(files_path))):
    words1,distance=vq(total_images_features[i],voc1) #vector quantization
    database_words.append(words1)
    for w in words1:
        im_features[i][w]+=1

In [20]:
im_features.shape #38images and their cluster frequency counts

(5, 100)

In [21]:
im_features[0].shape #first image with its cluster frequencies

(100,)

In [22]:
im_features[0] #no of occcurence of each cluster till 50 cluster with K array size

array([1481.,    0.,    0.,    0.,    6.,    0.,    0.,    0.,    0.,
          0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,
          0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,
         11.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,
          0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,
          0.,    2.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,
          0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,
          0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,
          0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,
          0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,
          0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,    0.,
          0.], dtype=float32)

In [23]:
database_words[0] #each and every keypoint grouped to one cluster.

array([0, 0, 0, ..., 0, 0, 0], dtype=int32)

### querry_image

In [ ]:
import os
#lets take a sample image
path = "gdrive/My Drive/copy_detection/few_images/1.jpg"
from PIL import Image
Image.open(path)

In [ ]:
from torchvision import transforms
file = Image.open(path)

data_transform = transforms.ToTensor()
img1 = data_transform(file)
op = model1(img1.unsqueeze(0))

In [ ]:
output = np.array(op.detach().numpy()[0]) #1,256,height,width

In [ ]:
output.shape #cfms generated

In [ ]:
import cv2
img = cv2.imread(path)

In [ ]:
orb = cv2.ORB_create(nfeatures=1500)
keypoints_orb, descriptors = orb.detectAndCompute(img, None)

keypoints = get_coordinates_and_radius(keypoints_orb)

In [ ]:
img.shape

In [ ]:
#Scaling CFM radius and coordinates
cfm_radius = []
cfm_coo = []
for key in keypoints:
    cfm_radius.append(np.float32(area_ratio*key[1]))

    cfm_x = (output.shape[1]*key[0][0])/img.shape[0]
    cfm_y = (output.shape[2]*key[0][1])/img.shape[1]
    cfm_coo.append((np.float32(cfm_x),np.float32(cfm_y)))

#Finding normalized radius from Minimum of 1 to B value
val = [key[1] for key in keypoints]    
x = np.array(cfm_radius)
b = (max(val)-min(val))/(min(val))
#     print(b)
a = 1
normalized_radius=(b-a)*((x-x.min())/(x.max()-x.min()))+a

In [ ]:
all_keypoint_features = []
output = np.array(op.detach().numpy()[0])
for cord,rad in zip(cfm_coo,normalized_radius):
    feature_map = []
    for fmap in range(len(output)):
        op = np.sum(output[fmap][int(cord[0]):int(cord[0])+int(rad),int(cord[1]):int(cord[1])+int(rad)])
        feature_map.append(op)
    op1 = np.array(feature_map).reshape(1,256)
    norm_final = sklearn.preprocessing.normalize(op1, norm='l2', axis=1, copy=True, return_norm=False)
    all_keypoint_features.append(norm_final)    
#     print(all_keypoint_features[0].shape)#(1,256)


In [ ]:
keypoint_feat = all_keypoint_features[0]
for keypoint in all_keypoint_features[1:]:
    keypoint_feat = np.vstack((keypoint_feat,keypoint))

In [ ]:
keypoint_feat.shape #324 keypoints only for a test image

### we got 256 local CNN features for al the 430 Keypoints. Now Generating CHE for all 430 keypoints

In [ ]:
img2 = cv2.drawKeypoints(img, keypoints_orb, None)

In [ ]:
import matplotlib.pyplot as plt
plt.figure(figsize = (10,12))
plt.imshow(img2)
plt.show()

In [ ]:
expanded_radius = []
for key,radius in keypoints:
    expand_r = int(int(radius/2)*3.2)
    expanded_radius.append(expand_r)

In [ ]:
cords = []
for cord,radius in keypoints:
    cords.append(cord)

In [ ]:
path = "gdrive/My Drive/copy_detection/copydays_original/32.jpg"

all_hash_values = []
for cord,ex_radius in zip(cords,expanded_radius):
    slices = get_blocks(cord,radius,path)
#     print(len(slices))
    if len(slices) == 25 or len(slices) == 20:
        grey_intensities = np.round(np.array([np.mean(i) for i in slices]).reshape(5,int(len(slices)/5)),2)    
    else:
        grey_intensities = np.round(np.array([np.mean(i) for i in slices]).reshape(4,int(np.ceil(16/5))),2)
    hash_val = get_hash_embedding(grey_intensities)
    all_hash_values.append(hash_val)

In [ ]:
len(all_hash_values) #324 hash points for our querry image

In [ ]:
import scipy
import random
random.seed(1234)

from scipy.cluster.vq import kmeans,vq #vq stands for vector quantization
k= 100
voc,variance=kmeans(keypoint_feat,k,100) #all keypoint features of all images

q_features=np.zeros((1,k),"float32")    
words,distance=vq(keypoint_feat,voc) #all keypoint features of single image
for w in words:
    q_features[0][w]+=1


In [ ]:
import sys
import numpy 
numpy.set_printoptions(threshold=sys.maxsize)

In [ ]:
q_features #It returns a vector of length n_clusters with its frequency count

### Final: data base image information

In [ ]:
img_feat.shape

In [ ]:
total_images_features[0].shape #local CNN features

In [ ]:
len(total_images_hash_values)

In [ ]:
len(total_images_hash_values[0]) #hash embeddings

In [ ]:
im_features.shape #VBOW

In [ ]:
voc1.shape #47 clusters

### Final: Querry image information

In [ ]:
keypoint_feat.shape #Local CNN Features

In [ ]:
len(all_hash_values) #hash embeddings

In [ ]:
q_features

In [ ]:
sum(q_features[0]) 

In [ ]:
voc.shape #shows clusters for our querry image

### Inverted index Creation 

In [ ]:
#generating filepath:im_features in dictionary

image_vocab = {} #filepath,vocab
for file_name,im_feature in zip(images,im_features):
    image_vocab[file_name] = list(im_feature)

In [ ]:
from collections import defaultdict

inv_indx = defaultdict(list) #defaultdict(list, {})

for imgidx,vocab in image_vocab.items():
    for idx,count in enumerate(vocab):
        if int(count) > 0:
            inv_indx[idx].append(imgidx)

 for all the images, which share the same cluster are represented as

    { 0:
    img1.png
    img2.png
    img3.png
     ,
     ,
     }

In [ ]:
inv_indx

In [ ]:
image_vocab['32.jpg']

In [ ]:
for indx,i in enumerate(im_features[37]):
    if i> 0:
        print(indx,i)

In [ ]:
im_features[37]

In [ ]:
database_words[37]

In [ ]:
from collections import defaultdict
inverted_indx = defaultdict(list) 


for image_id,feature,kp in zip(images,im_features,database_words): #101.jpg, im_feature,430kp
    for indx,count in enumerate(feature): #for every feature in im_features
        if int(count)>0: #if fequency value is >0
            no = int(image_id.split(".")[0]) # image no
            hashes = [{ind:total_images_hash_values[no-1][ind]}  for ind,j in enumerate(kp) if j == indx]
            inverted_indx[indx].append([image_id,hashes])

In [ ]:
inverted_indx

In [ ]:
len(inv_indx),len(inverted_indx) #cross check1

In [ ]:
for i,val in enumerate(database_words[0]):#cross check2
    if val == 0:
        print(i)

#### inverted_index for querry image

In [ ]:
#for querry image
#generating filepath:im_features in dictionary

q_image = ['32.jpg']
q_image_vocab = {} #filepath,vocab
for file_name,q_feature in zip(q_image,q_features):
    q_image_vocab[file_name] = list(q_feature)


In [ ]:
from collections import defaultdict

q_inv_indx = defaultdict(list) #defaultdict(list, {})

for imgidx,vocab in q_image_vocab.items():
    for idx,count in enumerate(vocab):
        if int(count) > 0:
            q_inv_indx[idx].append(imgidx)

In [ ]:
q_inv_indx

In [ ]:
len(all_hash_values),len(words),q_features

In [ ]:
from collections import defaultdict
q_inverted_indx = defaultdict(list) 


for indx,count in enumerate(q_features[0]): #for every feature in im_features
    if int(count)>0: #if fequency value is >0
        no = int(q_image[0].split(".")[0]) # image no
        hashes = [{ind:all_hash_values[ind]}  for ind,j in enumerate(words) if j == indx]
        q_inverted_indx[indx].append([q_image[0],hashes])

In [ ]:
database_words[0]

In [ ]:
import collections
from nltk import flatten
import sys
import math

all_kp =[]
for i in range(0,len(database_words)):
    all_kp.append(list(database_words[i]))

cat= flatten(all_kp) # all images keypoints to a single list :16664 keypoints
ctr = collections.Counter(cat)
final_dict=(dict(ctr))
final_dict = dict(collections.OrderedDict(sorted(final_dict.items())))
print(final_dict)

N= 45#len(final_dict)

IDF_database = []
for (term,term_frequency) in final_dict.items():
  term_IDF = math.log(float(N) / term_frequency)
  IDF_database.append([np.round(term_IDF,2), term])
IDF_database

In [ ]:
N

In [ ]:
inverted_indx.keys()

In [ ]:
inverted_indx[1][0][0] #cluster1, first images out of 6 images,imagename
inverted_indx[1][0][1] #cluster1, first images out of 6 images, kp and hash
inverted_indx[1][0][1][0].values() #first hash values

In [ ]:
from typing import List
def distance_hash_sequence(q_hash_sequence:List[str],d_hash_sequence:List[str])->float:
    M = 4
    N = 4
    distance = 0
    #calculating the distance
    for index in range(0,len(q_hash_sequence)):
        for q_hash_sequence_bit,d_hash_sequence_bit in zip(list(q_hash_sequence[index]),list(d_hash_sequence[index])):
            distance+=abs(int(q_hash_sequence_bit)-int(d_hash_sequence_bit))/(M*N*8)
    return distance

In [ ]:
dist_t = 0.35

final_res = {}
for i in q_inverted_indx.keys():                     #querry clusters
    for j in inverted_indx.keys():                   #database clusters
        if i==j:
            images = len(inverted_indx[i])#6 images to compare
            img_count = {}
            for image in range(images):             #in each image
                mini = min(len(inverted_indx[i][image][1]),len(q_inverted_indx[i][0][1])) #which one has less len
                for hs in range(mini):
                    a = list(q_inverted_indx[i][0][1][hs].values())[0]
                    b = list(inverted_indx[i][image][1][hs].values())[0]
                    dist = distance_hash_sequence(a,b)
                    if dist < dist_t:
                        if inverted_indx[i][image][0] not in img_count:
                            img_count[inverted_indx[i][image][0]]=0
                        if inverted_indx[i][image][0] in img_count:
                            img_count[inverted_indx[i][image][0]]+=1
    final_res[i]=img_count

In [ ]:
final_res

In [ ]:
final_res[46]

In [ ]:
for vbw  in range(len(q_inverted_indx)-1):
    for val,key in final_res[vbw].items():
        print(vbw,":",val,key*IDF_database[vbw][0])

In [ ]:
file = {}
for vbw  in range(len(q_inverted_indx)-1):
    for val,key in final_res[vbw].items():
#         print(val,key*IDF_database[vbw][0])
        if val not in file:
            file[val] = key*IDF_database[vbw][0]
        if val in file:
            file[val]+= key*IDF_database[vbw][0]
        

In [ ]:
sorted([file.values()])

In [ ]:
{k: v for k, v in sorted(file.items(), key=lambda item: item[1])}

## - Results are not good, entities to check On Monday are
####    - Model
####    - K-Means iterations,clusters
####    - Better Keypoint detecting algorithm
####    - IDF Generating